In [3]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
from kafka import KafkaProducer
import json
import time
import datetime

import sys
print(sys.executable)

# MySQL Connection
host = '192.168.3.102'
user = 'devturingmx'
password = quote_plus('des@rrollo')
database = 'sistema_monitoreo'

# Create SQLAlchemy engine
connection_url = f'mysql+pymysql://{user}:{password}@{host}/{database}'
engine = create_engine(connection_url)

# Initialize Kafka producer
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda x: json.dumps(x).encode('utf-8')
)

def fetch_and_send_data():
    try:
        with engine.connect() as connection:
            
            
            query_last_row = "SELECT * FROM fgenerador1_data ORDER BY id DESC LIMIT 1"
            # Añadimos text ya que sqlalchemy no es compatible directamente con cadenas de texto
            result = connection.execute(text(query_last_row)).fetchone()

            if result:
                # Modificamos data para convertir manualmete el timestamp por que json.dumps no puede convertirlo automaticamente
                data = {
                key: (value.isoformat() if isinstance(value, (datetime.datetime, datetime.date)) else value)
                for key, value in dict(result._mapping).items()
                }
                
                producer.send('rigcore-data', value=data)
                producer.flush()
                print(f"Sent data: {data}")
    except Exception as e:
        print(f"Error: {e}")

def main():
    while True:
        fetch_and_send_data()
        time.sleep(3)

if __name__ == "__main__":
    main()

/Users/ovillajc/Documents/Desarrollo/python/fast_api/rigcore_simulator/.venv/bin/python
Sent data: {'id': 4136, 'timestamp': '2025-06-20T16:39:49', 'rig_config_id': 3, 'session_timestamp': '2025-06-08T13:00:00', 'name_1': 'Status Word ', 'channel_01': 200.0, 'unit_01': 'unidad', 'name_2': 'Status Word 2', 'channel_02': 167363.0, 'unit_02': 'unidad', 'name_3': 'Engine Fuel Rate. Differencial', 'channel_03': 38.0, 'unit_03': 'l', 'name_4': 'Total Fuel Used Differencial (Clearabe)', 'channel_04': 414.0, 'unit_04': 'l', 'name_5': 'Flowmeter Hours Of Operation (Clearable)', 'channel_05': 21.0, 'unit_05': 'm/s', 'name_6': 'Flowmeter Hours Of Operation Suministro A-Gen (Clearable)', 'channel_06': 631.0, 'unit_06': 'unidad', 'name_7': 'Total Fuel Used  Retorno A-Gen (Clearabe)', 'channel_07': 300.0, 'unit_07': 'unidad', 'name_8': 'Flowmeter Hours Of Operation Retorno A-Gen (Clearable)', 'channel_08': 89.0, 'unit_08': 'unidad', 'name_9': 'Engine temperature Suministro', 'channel_09': 87.0, 'uni

KeyboardInterrupt: 

In [15]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
from kafka import KafkaProducer
import json
import time
import datetime

# MySQL Connection
host = '192.168.3.102'
user = 'devturingmx'
password = quote_plus('des@rrollo')
database = 'sistema_monitoreo'

# Create SQLAlchemy engine
connection_url = f'mysql+pymysql://{user}:{password}@{host}/{database}'
engine = create_engine(connection_url)

# Inicializar el productor de Kafka
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda x: json.dumps(x).encode('utf-8')
)


# 1: De volver lista de los nombres de todas las tablas de la base de datos
def fetch_all_table_names(connection):
    try:
        result = connection.execute(text("SHOW TABLES"))
        return [row[0] for row in result]
    except Exception as e:
        print(f"Error al obtener nombre de las tablas: {e}")
        return []


# 2: Obtener la ultima fila de una tabla por id descendente
def fetch_last_row(connection, table_name):
    try:
        result = connection.execute(
            text(f"SELECT * FROM {table_name} ORDER BY id DESC LIMIT 1")).fetchone()
        if result:
            return {
                key: (value.isoformat() if isinstance(
                    value, (datetime.datetime, datetime.date)) else value)
                for key, value in result._mapping.items()
            }
        else:
            return None
    except Exception as e:
        print(f"Error consultando la tabla {table_name} : {e}")
        return None


# 3: Obtner y mandar todas las ultimas filas de todas las tablas
def fetch_and_send_all_tables():
    try:
        with engine.connect() as connection:
            table_names = fetch_all_table_names(connection)
            for table in table_names:
                row_data = fetch_last_row(connection, table)
                if row_data:
                    payload = {
                        "table": table,
                        "data": row_data
                    }
                    producer.send('rigcore-data', value=payload)
                    print(f"[Kafka] Sent from {table}: {payload}")
            producer.flush()
    except Exception as e:
        print(f"Error en la conexion o envio de data: {e}")


def main():
    print("Iniciando monitoreo...")
    try:
        while True:
            fetch_and_send_all_tables()
            time.sleep(3)
    except KeyboardInterrupt:
        print("Monitoreo detenido manualmente.")


main()

Iniciando monitoreo...
[Kafka] Sent from calculos_config: {'table': 'calculos_config', 'data': {'id': 15, 'id_codigo': 'codigo_01', 'session_timestamp': '2025-06-08T13:00:00', 'nombre': 'suma', 'descripcion': 'Descripción del cálculo1', 'formula': 'var1+var2', 'timestamp_created': '2025-06-15T02:55:35'}}
[Kafka] Sent from calculos_resultado: {'table': 'calculos_resultado', 'data': {'id': 909, 'timestamp': '2025-06-14T19:27:24', 'session_timestamp': '2025-06-08T13:00:00', 'codigo_01': 'gene', 'resultado_01': '0.0000', 'unidad_01': '', 'codigo_02': None, 'resultado_02': None, 'unidad_02': None, 'codigo_03': None, 'resultado_03': None, 'unidad_03': None, 'codigo_04': None, 'resultado_04': None, 'unidad_04': None, 'codigo_05': None, 'resultado_05': None, 'unidad_05': None, 'codigo_06': None, 'resultado_06': None, 'unidad_06': None, 'codigo_07': None, 'resultado_07': None, 'unidad_07': None, 'codigo_08': None, 'resultado_08': None, 'unidad_08': None, 'codigo_09': None, 'resultado_09': None,

In [ ]:
from kafka import KafkaConsumer
import json
import time

# Configuracion del consumidor
consumer = KafkaConsumer(
    'rigcore-data',
    bootstrap_servers='localhost:9092',
    auto_offset_reset='latest',
    enable_auto_commit=True,     
    group_id='rigcore-consumer-group',
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

print("Escuchando mensajes en el topic 'rigcore-data'...")

try:
    for message in consumer:
        data = message.value
        print(f"Mensaje recibido: {data}")
        time.sleep(1)
except KeyboardInterrupt:
    print("\nLectura interrumpida por el usuario.")
finally:
    consumer.close()
    print("Consumer cerrado correctamente.")

Escuchando mensajes en el topic 'rigcore-data'...
Mensaje recibido: {'id': 4136, 'timestamp': '2025-06-20T16:39:49', 'rig_config_id': 3, 'session_timestamp': '2025-06-08T13:00:00', 'name_1': 'Status Word ', 'channel_01': 200.0, 'unit_01': 'unidad', 'name_2': 'Status Word 2', 'channel_02': 167363.0, 'unit_02': 'unidad', 'name_3': 'Engine Fuel Rate. Differencial', 'channel_03': 38.0, 'unit_03': 'l', 'name_4': 'Total Fuel Used Differencial (Clearabe)', 'channel_04': 414.0, 'unit_04': 'l', 'name_5': 'Flowmeter Hours Of Operation (Clearable)', 'channel_05': 21.0, 'unit_05': 'm/s', 'name_6': 'Flowmeter Hours Of Operation Suministro A-Gen (Clearable)', 'channel_06': 631.0, 'unit_06': 'unidad', 'name_7': 'Total Fuel Used  Retorno A-Gen (Clearabe)', 'channel_07': 300.0, 'unit_07': 'unidad', 'name_8': 'Flowmeter Hours Of Operation Retorno A-Gen (Clearable)', 'channel_08': 89.0, 'unit_08': 'unidad', 'name_9': 'Engine temperature Suministro', 'channel_09': 87.0, 'unit_09': 'unidad', 'name_10': 'En